In [1]:
# 1. 데이터 확보
import zipfile
filepath = 'data/glaucoma.zip'
with zipfile.ZipFile(filepath, 'r') as f:
    f.extractall('data/')

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(
    rotation_range=20, # 최대 회전 각도
    width_shift_range=0.2, # 가로 방향 -20%~20% 이동 정도
    height_shift_range=0.2, # 상하 방향 이동 정도
    horizontal_flip=True, # 수평뒤집기
    vertical_flip=True,) # 수직뒤집기
train_data = train_generator.flow_from_directory(
    'data/glaucoma/train',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse' # 종속변수에 원핫인코딩 안 함
)
test_generator = ImageDataGenerator()
test_data = test_generator.flow_from_directory(
    'data/glaucoma/test',
    target_size=(224,224),
    batch_size=4,
    class_mode='sparse'
)

Found 828 images belonging to 3 classes.
Found 320 images belonging to 3 classes.


In [3]:
# 모델구성
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
import numpy as np
from tensorflow.keras.models import Sequential # ResNet50의 Dense층에 사용자 추가
from tensorflow.keras.layers import Flatten, Dense
import cv2 # 이미지를 넘파이로 읽어오기

In [4]:
resnet_model = ResNet50(
    input_shape=(224,224,3),
    include_top=False,) # 컨볼루션과 폴링 레이어만
resnet_model.trainable=True

In [5]:
# model = Sequential()
# model.add(resnet_model)
# model.add(Flatten())
# model.add(Dense(units=1024, activation='relu'))
# model.add(Dense(units=3, activation='softmax'))
model = Sequential([
    resnet_model,
    Flatten(),
    Dense(units=1024,activation='relu'),
    Dense(units=3, activation='softmax')
])

model.summary()

# 학습 시 필요한 용량
# 1. 파라미터 사용 메모리 사용량 : 126,352,259 * 4 byte / 1024의 제곱 = 482MB
# 2. 배치사이즈=3일 경우 3*(224*224*4)*4 byte / 1024의 제곱 = 19MB
# 각 layer층 마다 활성화 값

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 1024)              102761472 
                                                                 
 dense_1 (Dense)             (None, 3)                 3075      
                                                                 
Total params: 126,352,259
Trainable params: 126,299,139
Non-trainable params: 53,120
_________________________________________________________________


In [6]:
# 모델 학습 설정
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# 모델 학습
model.fit(train_data, epochs=20, validation_data=test_data, batch_size=1)

Epoch 1/20


ResourceExhaustedError: Graph execution error:

Detected at node 'sequential/resnet50/conv4_block6_3_conv/Conv2D' defined at (most recent call last):
    File "C:\Users\4545\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\4545\anaconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\4545\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\4545\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\4545\anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "C:\Users\4545\anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\4545\anaconda3\lib\asyncio\base_events.py", line 603, in run_forever
      

In [10]:
test_img = cv2.imread('data/test.png')
test_img = cv2.resize(test_img, dsize=(224,224))
test_img.shape

(224, 224, 3)